In [1]:
import pandas as pd

In [19]:
import json
import pandas as pd
from datetime import datetime

# Load JSON data
with open('MetaAds\poloshirts_meta_ads.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Flatten nested structures
# json_normalize will automatically expand nested dicts; lists remain as lists
df = pd.json_normalize(
    data,
    sep='.'  # use dot to separate nested keys
)

# Drop columns with all missing values
df.dropna(axis=1, how='all', inplace=True)

# Convert Unix timestamps to datetime for start_date and end_date
for col in ['start_date', 'end_date']:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], unit='s', errors='coerce')

# Handle missing values
# For numeric columns, fill missing with 0
num_cols = df.select_dtypes(include=['number']).columns
df[num_cols] = df[num_cols].fillna(0)

# For object/string columns, fill missing with empty string
obj_cols = df.select_dtypes(include=['object']).columns
df[obj_cols] = df[obj_cols].fillna('')

# Example: if there are list columns you may want to convert them to strings
list_cols = [c for c in df.columns if df[c].apply(lambda x: isinstance(x, list)).any()]
for col in list_cols:
    df[col] = df[col].apply(lambda x: ','.join(map(str, x)) if isinstance(x, list) else x)

# Save to CSV
output_csv = '268_meta_video_Polo_Shirts_ads_clean.csv'
df.to_csv(output_csv, index=False, encoding='utf-8')

print(f"Cleaned CSV saved to {output_csv}")


Cleaned CSV saved to 268_meta_video_Polo_Shirts_ads_clean.csv


In [21]:
meta = pd.read_csv('268_meta_video_Polo_Shirts_ads_clean.csv')
meta

,ad_archive_id,page_id,page_is_deleted,page_name,archive_types,categories,collation_count,collation_id,contains_digital_created_media,contains_sensitive_content,...,regional_regulation_data.finserv.is_deemed_finserv,regional_regulation_data.finserv.is_limited_delivery,regional_regulation_data.tw_anti_scam.is_limited_delivery,snapshot.branded_content.current_page_name,snapshot.branded_content.page_categories,snapshot.branded_content.page_id,snapshot.branded_content.page_is_deleted,snapshot.branded_content.page_name,snapshot.branded_content.page_profile_pic_url,snapshot.branded_content.page_profile_uri
0,4489700794589556,100467538566051,False,Brand BD,NaN,UNKNOWN,1.0,5.483568e+14,False,False,...,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3992827137602264,439613319237701,False,1986 golf clothing factory,NaN,UNKNOWN,1.0,1.024033e+15,False,False,...,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1686116558943568,285426894644881,False,QianDao Sportwear Manufacturer,NaN,UNKNOWN,1.0,9.900566e+14,False,False,...,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1244477810433871,523807967485266,False,GriffinGods,NaN,UNKNOWN,0.0,NaN,False,False,...,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1265694751786619,112887030479737,False,Orihica Global,NaN,UNKNOWN,1.0,1.043923e+15,False,False,...,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,672300445538589,203422283755935,False,DVF MALL,NaN,UNKNOWN,1.0,6.478541e+14,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
264,562122386549343,237397950365932,False,Saint Terror,NaN,UNKNOWN,1.0,2.293308e+15,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
265,666056483020362,339907835879320,False,Ganesh Garments,NaN,UNKNOWN,1.0,1.248487e+15,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
266,724879253301071,114750993248952,False,EQL,NaN,UNKNOWN,1.0,4.123661e+15,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
import pandas as pd
from datetime import datetime

# Load the CSV
meta = pd.read_csv('268_meta_video_Polo_Shirts_ads_clean.csv')

# Drop columns that have all NaN values
meta.dropna(axis=1, how='all', inplace=True)

# Ensure 'start_date' is in datetime format
if 'start_date' in meta.columns:
    meta['start_date'] = pd.to_datetime(meta['start_date'], errors='coerce')

# Calculate days since start
today = pd.Timestamp.today()
meta['days_since_start'] = (today - meta['start_date']).dt.days

# Preview the cleaned DataFrame
print(meta.head())


      ad_archive_id          page_id  page_is_deleted  \
0  4489700794589556  100467538566051            False   
1  3992827137602264  439613319237701            False   
2  1686116558943568  285426894644881            False   
3  1244477810433871  523807967485266            False   
4  1265694751786619  112887030479737            False   

                        page_name categories  collation_count  collation_id  \
0                        Brand BD    UNKNOWN              1.0  5.483568e+14   
1      1986 golf clothing factory    UNKNOWN              1.0  1.024033e+15   
2  QianDao Sportwear Manufacturer    UNKNOWN              1.0  9.900566e+14   
3                     GriffinGods    UNKNOWN              0.0           NaN   
4                  Orihica Global    UNKNOWN              1.0  1.043923e+15   

   contains_digital_created_media  contains_sensitive_content  \
0                           False                       False   
1                           False                   

In [77]:
meta['snapshot'].nunique()

941

In [23]:
meta[meta['days_since_start']  >= 14]

,ad_archive_id,page_id,page_is_deleted,page_name,categories,collation_count,collation_id,contains_digital_created_media,contains_sensitive_content,end_date,...,regional_regulation_data.finserv.is_limited_delivery,regional_regulation_data.tw_anti_scam.is_limited_delivery,snapshot.branded_content.current_page_name,snapshot.branded_content.page_categories,snapshot.branded_content.page_id,snapshot.branded_content.page_is_deleted,snapshot.branded_content.page_name,snapshot.branded_content.page_profile_pic_url,snapshot.branded_content.page_profile_uri,days_since_start


In [50]:
meta['snapshot.branded_content.page_categories'] #.nunique()

0                NaN
1                NaN
2                NaN
3                NaN
4                NaN
           ...      
995              NaN
996              NaN
997              NaN
998              NaN
999    Personal blog
Name: snapshot.branded_content.page_categories, Length: 1000, dtype: object

In [78]:
columns_to_keep = ['page_name','snapshot.page_profile_picture_url','snapshot.body.text','snapshot.caption','snapshot.cta_text','snapshot.cta_type', 'snapshot.link_description',
                   'snapshot.link_url','snapshot.page_categories','snapshot.page_like_count','snapshot.title','snapshot.videos',
                    'days_since_start']

# Filter the DataFrame
meta = meta[columns_to_keep]

# 2. Drop rows where snapshot.videos is NaN
meta = meta.dropna(subset=['snapshot.videos'])

# 3. Drop rows where snapshot.videos is “empty”—
#    If it’s a list-like object:
meta = meta[meta['snapshot.videos'].apply(lambda vids: bool(vids))]



In [61]:
meta['snapshot.page_like_count'].nunique()

329

In [90]:
meta[ (meta['days_since_start'] >= 14) &
    (meta['snapshot.page_like_count']  > 18000) ]

,page_name,snapshot.page_profile_picture_url,snapshot.body.text,snapshot.caption,snapshot.cta_text,snapshot.cta_type,snapshot.link_description,snapshot.link_url,snapshot.page_categories,snapshot.page_like_count,snapshot.title,snapshot.videos,days_since_start
714,Collars & Co.,https://scontent.fkul10-1.fna.fbcdn.net/v/t39....,We didn’t invent the polo. We just made it bet...,collarsandco.com,Shop now,SHOP_NOW,NaN,https://collarsandco.com/collections/mens-polo...,Clothing (Brand),21128,This Collar Is Rewriting the Rules.,{'video_hd_url': 'https://video.fkul15-1.fna.f...,14
716,Montivo,https://scontent.fkul15-1.fna.fbcdn.net/v/t39....,"Men colour block slim fit polos\n\n[Polos, men...",montivo.pk,Shop now,SHOP_NOW,NaN,https://montivo.pk/collections/men-polos,Clothing (Brand),31750,NaN,{'video_hd_url': 'https://video.fkul15-1.fna.f...,14
717,Collars & Co.,https://scontent.fkul15-1.fna.fbcdn.net/v/t39....,We didn’t invent the polo—we made it better.\n...,collarsandco.com,Shop now,SHOP_NOW,NaN,https://collarsandco.com/collections/semi-spre...,Clothing (Brand),21128,The Polo That Replaces Your Dress Shirt—Shop Now.,{'video_hd_url': 'https://video.fkul15-1.fna.f...,14
729,Patrick Bourke Menswear,https://scontent.fkul10-1.fna.fbcdn.net/v/t39....,𝐅𝐫𝐞𝐬𝐡 𝐃𝐫𝐨𝐩: 𝐓𝐨𝐦𝐦𝐲 𝐇𝐢𝐥𝐟𝐢𝐠𝐞𝐫 𝐏𝐨𝐥𝐨 𝐒𝐡𝐢𝐫𝐭𝐬\n\nChec...,patrickbourkemenswear.ie,Shop now,SHOP_NOW,NaN,https://patrickbourkemenswear.ie/collections/t...,Men's Clothing,18852,The Icon Returns,{'video_hd_url': 'https://video.fkul10-1.fna.f...,15
738,Ivan - Gentlemen Wear,https://scontent-dfw5-2.xx.fbcdn.net/v/t39.354...,Polo ကို High Class ဖြစ်တာမျိုး ဝတ်ချင်တာလား\n...,NaN,Send message,MESSAGE_PAGE,Men's clothing store,NaN,Men's Clothing,154719,Ivan - Gentlemen Wear,{'video_hd_url': 'https://video-dfw5-2.xx.fbcd...,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,Hanley.ie,https://scontent.flhr1-2.fna.fbcdn.net/v/t39.3...,Join Paul as he presents GANT's Spring/Summer ...,hanley.ie,Shop now,SHOP_NOW,NaN,https://hanley.ie/collections/gant-clothing,Men's Clothing,22088,NaN,{'video_hd_url': 'https://video.flhr1-2.fna.fb...,59
995,Boom Outfits,https://scontent.flhr1-1.fna.fbcdn.net/v/t39.3...,"🔥 World-Class Brands, এখন একদম হাতের নাগালে!\n...",NaN,Like Page,LIKE_PAGE,NaN,NaN,Clothing (Brand),19473,NaN,{'video_hd_url': 'https://video.flhr1-1.fna.fb...,60
996,Tie Bar,https://scontent.flhr1-1.fna.fbcdn.net/v/t39.3...,From classic office style to something a bit m...,www.thetiebar.com,Shop now,SHOP_NOW,NaN,https://www.thetiebar.com/c/spring-style?uc_pa...,Retail company,77013,3 Ways to style the Lakeshore Chalk Stripe Suit.,{'video_hd_url': 'https://video.flhr1-2.fna.fb...,61
998,Beyoung.in,https://scontent.fcnf1-1.fna.fbcdn.net/v/t39.3...,Last chance to grab this drop! Super breathabl...,www.beyoung.in,Shop now,SHOP_NOW,Free Shipping | COD Available,https://www.beyoung.in/polo-t-shirts?utm_sourc...,Website,65521,🎯 The Perfect Polo at the Perfect Price!,{'video_hd_url': 'https://video.fcnf1-1.fna.fb...,63


In [91]:
meta[ (meta['days_since_start'] >= 14) &
    (meta['snapshot.page_like_count'] > 18000) ].to_csv('68_filtered_meta_video_Polo_shirts_ads.csv' , index= False)

In [92]:
import pandas as pd
import json

# 1. Read your CSV
filt = pd.read_csv('68_filtered_meta_video_Polo_shirts_ads.csv')

# 2. Convert to a list of dicts (one dict per row)
records = filt.to_dict(orient='records')

# 3. Write out pretty-printed JSON, one record at a time
with open('68_filtered_meta_pretty.json', 'w', encoding='utf-8') as fout:
    for rec in records:
        json.dump(rec, fout, ensure_ascii=False, indent=4)
        fout.write('\n')  # blank line between records (optional)


In [94]:
import pandas as pd
import json
import ast

# 1. Load all JSON objects from the pretty-printed JSON file
input_path = '68_filtered_meta_pretty.json'
text = open(input_path, 'r', encoding='utf-8').read()
decoder = json.JSONDecoder()
records = []
idx = 0
length = len(text)

while idx < length:
    while idx < length and text[idx].isspace():
        idx += 1
    if idx >= length:
        break
    obj, end = decoder.raw_decode(text, idx)
    records.append(obj)
    idx = end

# 2. Load into a DataFrame
df = pd.DataFrame(records)

# 3. Parse 'snapshot.videos' string to dictionary
df['snapshot.videos'] = df['snapshot.videos'].apply(ast.literal_eval)

# 4. Extract video_hd_url
df['video_hd_url'] = df['snapshot.videos'].apply(lambda d: d.get('video_hd_url', '').strip())

# 5. Remove rows where video_hd_url is empty
df = df[df['video_hd_url'] != ""]

# 6. Group by video URL and aggregate
agg = df.groupby('video_hd_url').agg(
    count=('video_hd_url', 'size'),
    days_since_start=('days_since_start', 'max'),
    page_name=('page_name', 'first'),
    snapshot_page_profile_picture_url=('snapshot.page_profile_picture_url', 'first'),
    snapshot_body_text=('snapshot.body.text', 'first'),
    snapshot_caption=('snapshot.caption', 'first'),
    snapshot_cta_text=('snapshot.cta_text', 'first'),
    snapshot_cta_type=('snapshot.cta_type', 'first'),
    snapshot_link_description=('snapshot.link_description', 'first'),
    snapshot_link_url=('snapshot.link_url', 'first'),
    snapshot_page_categories=('snapshot.page_categories', 'first'),
    snapshot_page_like_count=('snapshot.page_like_count', 'first'),
    snapshot_title=('snapshot.title', 'first'),
    video_preview_image_url=('snapshot.videos', lambda d: d.get('video_preview_image_url', ''))
).reset_index()

# 7. Sort by count DESC, then days_since_start DESC
agg = agg.sort_values(by=['count', 'days_since_start'], ascending=[False, False])

# 8. Re-nest 'snapshot.videos'
agg['snapshot.videos'] = agg.apply(lambda r: {
    'video_hd_url': r['video_hd_url'],
    'video_preview_image_url': r['video_preview_image_url']
}, axis=1)

# 9. Drop helper columns
agg.drop(columns=['video_hd_url', 'video_preview_image_url'], inplace=True)

# 10. Save output to pretty NDJSON
output_path = '62_filtered_meta_aggregated.json'
with open(output_path, 'w', encoding='utf-8') as fout:
    for rec in agg.to_dict(orient='records'):
        json.dump(rec, fout, ensure_ascii=False, indent=4)
        fout.write('\n\n')

print(f"✅ {len(agg)} unique ads written to: {output_path}")


✅ 62 unique ads written to: 62_filtered_meta_aggregated.json


In [95]:
import pandas as pd
import json
import ast

# 1. Load all JSON objects from the pretty-formatted input
input_path = '68_filtered_meta_pretty.json'
text = open(input_path, 'r', encoding='utf-8').read()
decoder = json.JSONDecoder()
records = []
idx = 0
length = len(text)

while idx < length:
    while idx < length and text[idx].isspace():
        idx += 1
    if idx >= length:
        break
    obj, end = decoder.raw_decode(text, idx)
    records.append(obj)
    idx = end

# 2. Convert to DataFrame
df = pd.DataFrame(records)

# 3. Convert snapshot.videos string to dict
df['snapshot.videos'] = df['snapshot.videos'].apply(ast.literal_eval)

# 4. Extract video_hd_url
df['video_hd_url'] = df['snapshot.videos'].apply(lambda d: d.get('video_hd_url', '').strip())

# 5. Drop rows with empty video_hd_url
df = df[df['video_hd_url'] != ""]

# 6. Group by video URL and aggregate
agg = df.groupby('video_hd_url').agg(
    count=('video_hd_url', 'size'),
    days_since_start=('days_since_start', 'max'),
    page_name=('page_name', 'first'),
    snapshot_page_profile_picture_url=('snapshot.page_profile_picture_url', 'first'),
    snapshot_body_text=('snapshot.body.text', 'first'),
    snapshot_caption=('snapshot.caption', 'first'),
    snapshot_cta_text=('snapshot.cta_text', 'first'),
    snapshot_cta_type=('snapshot.cta_type', 'first'),
    snapshot_link_description=('snapshot.link_description', 'first'),
    snapshot_link_url=('snapshot.link_url', 'first'),
    snapshot_page_categories=('snapshot.page_categories', 'first'),
    snapshot_page_like_count=('snapshot.page_like_count', 'first'),
    snapshot_title=('snapshot.title', 'first'),
    video_preview_image_url=('snapshot.videos', lambda d: d.get('video_preview_image_url', ''))
).reset_index()

# 7. Sort by days_running, then count, then likes
agg = agg.sort_values(
    by=['days_since_start', 'count', 'snapshot_page_like_count'],
    ascending=[False, False, False]
)

# 8. Keep top 40 ads only
agg = agg.head(40)

# 9. Recreate snapshot.videos dict
agg['snapshot.videos'] = agg.apply(lambda r: {
    'video_hd_url': r['video_hd_url'],
    'video_preview_image_url': r['video_preview_image_url']
}, axis=1)

# 10. Drop helper columns
agg.drop(columns=['video_hd_url', 'video_preview_image_url'], inplace=True)

# 11. Write to pretty NDJSON
output_path = 'top40_sorted__meta_ads.json'
with open(output_path, 'w', encoding='utf-8') as fout:
    for rec in agg.to_dict(orient='records'):
        json.dump(rec, fout, ensure_ascii=False, indent=4)
        fout.write('\n\n')

print(f"✅ Top 40 ads saved to: {output_path}")


✅ Top 40 ads saved to: top40_sorted__meta_ads.json


In [72]:
filt = pd.read_csv('290_filtered_meta_video_Polo_shirts_ads.csv')
print(filt.head())

                    page_name  \
0                Austin Style   
1  Apparel & Accessories GPBX   
2  Apparel & Accessories GPBX   
3             FlashingBy Shop   
4               Markus Baulig   

                   snapshot.page_profile_picture_url  \
0  https://scontent-atl3-1.xx.fbcdn.net/v/t39.354...   
1  https://scontent-atl3-1.xx.fbcdn.net/v/t39.354...   
2  https://scontent-atl3-2.xx.fbcdn.net/v/t39.354...   
3  https://scontent-atl3-2.xx.fbcdn.net/v/t39.354...   
4  https://scontent-atl3-2.xx.fbcdn.net/v/t39.354...   

                                  snapshot.body.text       snapshot.caption  \
0  Did you know? Polo shirts make you 10x more at...  www.austinstyle.store   
1  5 Collar T-shirts with Pocket! Only @599/-\nGe...           step2buy.com   
2  5 Collar T-shirts with Pocket! Only @599/-\nGe...           step2buy.com   
3  Rule Your Wardrobe With Lavish Branded Plaid P...         flashingby.com   
4  Wenn du dich wie ein Billig-Kunde verhältst, z...       andreasbau

In [73]:
filt['snapshot.videos']

0      {'video_hd_url': 'https://video-atl3-1.xx.fbcd...
1      {'video_hd_url': 'https://video-atl3-1.xx.fbcd...
2      {'video_hd_url': 'https://video-atl3-2.xx.fbcd...
3      {'video_hd_url': 'https://video-atl3-2.xx.fbcd...
4                                                    NaN
                             ...                        
285    {'video_hd_url': 'https://video.flhr1-2.fna.fb...
286    {'video_hd_url': 'https://video.flhr1-1.fna.fb...
287    {'video_hd_url': 'https://video.flhr1-2.fna.fb...
288    {'video_hd_url': 'https://video.fcnf1-1.fna.fb...
289    {'video_hd_url': 'https://video.fcnf1-1.fna.fb...
Name: snapshot.videos, Length: 290, dtype: object

In [54]:
meta.shape

(1000, 14)

In [26]:
meta['categories'].nunique()

2

In [25]:
meta.to_csv('283_filtered_meta_video_Polo_shirts_ads.csv' , index= False)

In [57]:
meta['snapshot.videos']

0      {'video_hd_url': 'https://video-atl3-2.xx.fbcd...
1      {'video_hd_url': 'https://video-atl3-1.xx.fbcd...
2      {'video_hd_url': 'https://video-atl3-1.xx.fbcd...
3      {'video_hd_url': 'https://video-atl3-1.xx.fbcd...
4      {'video_hd_url': 'https://video-atl3-1.xx.fbcd...
                             ...                        
995    {'video_hd_url': 'https://video.flhr1-1.fna.fb...
996    {'video_hd_url': 'https://video.flhr1-2.fna.fb...
997    {'video_hd_url': 'https://video.fcnf1-1.fna.fb...
998    {'video_hd_url': 'https://video.fcnf1-1.fna.fb...
999    {'video_hd_url': 'https://video.fcnf1-1.fna.fb...
Name: snapshot.videos, Length: 1000, dtype: object

In [2]:
df = pd.read_csv('filtered_Polo_shirts_ads.csv')

In [5]:
df.head(15)

,caption,video_hd_url,video_preview_image_url,video_sd_url,watermarked_video_hd_url,watermarked_video_sd_url,start_date,end_date,title,text,page_like_count,post_url,cta_text,cta_type,days_since_start
0,NaN,https://video-lhr8-2.xx.fbcdn.net/v/t42.1790-2...,https://scontent-lhr6-2.xx.fbcdn.net/v/t39.354...,https://video-lhr6-1.xx.fbcdn.net/v/t42.1790-2...,NaN,NaN,2025-04-24 07:00:00,2025-05-08 07:00:00,From Gym to Sun – Ultimate Comfort + UPF 50+ B...,Gear up with Technosport – For the Street & th...,45878,"fbgeo://11.675192220538957,78.15200954675674,""...",Get directions,GET_DIRECTIONS,14
1,NaN,https://video.fpoa44-1.fna.fbcdn.net/v/t42.179...,https://scontent.fpoa44-1.fna.fbcdn.net/v/t39....,https://video.fpoa44-1.fna.fbcdn.net/v/t42.179...,NaN,NaN,2025-04-23 07:00:00,2025-05-07 07:00:00,Grab attention in style!,Get your pack of 5 collar T-shirts with pocket...,1576,NaN,Call now,CALL_NOW,14
2,hektor.in,https://video.fpoa44-1.fna.fbcdn.net/v/t42.179...,https://scontent.fpoa44-1.fna.fbcdn.net/v/t39....,https://video.fpoa44-1.fna.fbcdn.net/v/t42.179...,NaN,NaN,2025-04-23 07:00:00,2025-05-08 07:00:00,"Effortless Style, Premium Comfort",Step into style with Hektor's premium polo t-s...,6,https://hektor.in/,Shop now,SHOP_NOW,15
3,hektor.in,https://video.fpoa44-1.fna.fbcdn.net/v/t42.179...,https://scontent.fpoa44-1.fna.fbcdn.net/v/t39....,https://video.fpoa44-1.fna.fbcdn.net/v/t42.179...,NaN,NaN,2025-04-23 07:00:00,2025-05-08 07:00:00,"Effortless Style, Premium Comfort",Step into style with Hektor's premium polo t-s...,6,https://hektor.in/,Shop now,SHOP_NOW,15
4,instagram.com,https://video.fpoa44-1.fna.fbcdn.net/v/t42.179...,https://scontent.fpoa44-1.fna.fbcdn.net/v/t39....,https://video.fpoa44-1.fna.fbcdn.net/v/t42.179...,NaN,NaN,2025-04-23 07:00:00,2025-05-08 07:00:00,NaN,100% Original Showroom Wale Brand Yaha Saste M...,1465,http://instagram.com/tushitaclothingco_warehouse,Visit Instagram profile,VIEW_INSTAGRAM_PROFILE,15
5,www.bonheurkids.in,https://video.fpoa44-1.fna.fbcdn.net/v/t42.179...,https://scontent.fpoa44-1.fna.fbcdn.net/v/t39....,https://video.fpoa44-1.fna.fbcdn.net/v/t42.179...,NaN,NaN,2025-04-21 07:00:00,2025-05-08 07:00:00,"Soft, Playful, and Stylish",Upgrade your kid's wardrobe with the perfect b...,221,https://www.bonheurkids.in/collections/polo-t-...,Shop now,SHOP_NOW,17
6,steekstyle.com,https://video.fpoa44-1.fna.fbcdn.net/v/t42.179...,https://scontent.fpoa44-1.fna.fbcdn.net/v/t39....,https://video.fpoa44-1.fna.fbcdn.net/v/t42.179...,NaN,NaN,2025-04-22 07:00:00,2025-05-07 07:00:00,Perfect Plain T-Shirt,Classic. Versatile. Timeless.\n\nUpgrade your ...,265,https://steekstyle.com/collections/polo-t-shir...,Shop now,SHOP_NOW,15
7,steekstyle.com,https://video.fpoa44-1.fna.fbcdn.net/v/t42.179...,https://scontent.fpoa44-1.fna.fbcdn.net/v/t39....,https://video.fpoa44-1.fna.fbcdn.net/v/t42.179...,NaN,NaN,2025-04-23 07:00:00,2025-05-08 07:00:00,Perfect Polo -Shirt,Classic. Versatile. Timeless.\n\nUpgrade your ...,265,https://steekstyle.com/collections/polo-t-shir...,Shop now,SHOP_NOW,15
8,www.vellure.in,https://video.ftun20-1.fna.fbcdn.net/v/t42.179...,https://scontent.ftun20-1.fna.fbcdn.net/v/t39....,https://video.ftun20-1.fna.fbcdn.net/v/t42.179...,NaN,NaN,2025-04-20 07:00:00,2025-05-07 07:00:00,NaN,"Your search for summer polos ends here — soft,...",166,https://www.vellure.in/collections/old-money-k...,Shop now,SHOP_NOW,17
9,www.vellure.in,https://video.ftun20-1.fna.fbcdn.net/v/t42.179...,https://scontent.ftun20-1.fna.fbcdn.net/v/t39....,https://video.ftun20-1.fna.fbcdn.net/v/t42.179...,NaN,NaN,2025-04-20 07:00:00,2025-05-07 07:00:00,NaN,There’s something about a well-fitted polo\nTa...,166,https://www.vellure.in/collections/old-money-k...,Shop now,SHOP_NOW,17


In [13]:
df1 = pd.read_csv('filtered_Polo_shirts_ads_tagged.csv')

In [14]:
df1.head()

,caption,video_hd_url,video_preview_image_url,video_sd_url,watermarked_video_hd_url,watermarked_video_sd_url,start_date,end_date,title,text,...,cta_text,cta_type,days_since_start,hierarchy_tag,storyline_tag,hook_tag,cta_tag,icp_tag,actor_tag,video_file_name
0,NaN,https://video-lhr8-2.xx.fbcdn.net/v/t42.1790-2...,https://scontent-lhr6-2.xx.fbcdn.net/v/t39.354...,https://video-lhr6-1.xx.fbcdn.net/v/t42.1790-2...,NaN,NaN,2025-04-24 07:00:00,2025-05-08 07:00:00,From Gym to Sun – Ultimate Comfort + UPF 50+ B...,Gear up with Technosport – For the Street & th...,...,Get directions,GET_DIRECTIONS,14,none,none,none,none,none,none,tmp0ibeoajf.xx&_nc_gid=SpfjdeXNqk7fZweKWHkmfQ&...
1,NaN,https://video.fpoa44-1.fna.fbcdn.net/v/t42.179...,https://scontent.fpoa44-1.fna.fbcdn.net/v/t39....,https://video.fpoa44-1.fna.fbcdn.net/v/t42.179...,NaN,NaN,2025-04-23 07:00:00,2025-05-07 07:00:00,Grab attention in style!,Get your pack of 5 collar T-shirts with pocket...,...,Call now,CALL_NOW,14,none,none,none,none,none,none,tmpqtsa5913.fna&_nc_gid=spdEaB2Lb0gw966KoBaCBA...
2,hektor.in,https://video.fpoa44-1.fna.fbcdn.net/v/t42.179...,https://scontent.fpoa44-1.fna.fbcdn.net/v/t39....,https://video.fpoa44-1.fna.fbcdn.net/v/t42.179...,NaN,NaN,2025-04-23 07:00:00,2025-05-08 07:00:00,"Effortless Style, Premium Comfort",Step into style with Hektor's premium polo t-s...,...,Shop now,SHOP_NOW,15,none,none,none,none,none,none,tmpwas4dgzp.fna&_nc_gid=spdEaB2Lb0gw966KoBaCBA...
3,hektor.in,https://video.fpoa44-1.fna.fbcdn.net/v/t42.179...,https://scontent.fpoa44-1.fna.fbcdn.net/v/t39....,https://video.fpoa44-1.fna.fbcdn.net/v/t42.179...,NaN,NaN,2025-04-23 07:00:00,2025-05-08 07:00:00,"Effortless Style, Premium Comfort",Step into style with Hektor's premium polo t-s...,...,Shop now,SHOP_NOW,15,none,none,none,none,none,none,tmp7mmte77c.fna&_nc_gid=spdEaB2Lb0gw966KoBaCBA...
4,instagram.com,https://video.fpoa44-1.fna.fbcdn.net/v/t42.179...,https://scontent.fpoa44-1.fna.fbcdn.net/v/t39....,https://video.fpoa44-1.fna.fbcdn.net/v/t42.179...,NaN,NaN,2025-04-23 07:00:00,2025-05-08 07:00:00,NaN,100% Original Showroom Wale Brand Yaha Saste M...,...,Visit Instagram profile,VIEW_INSTAGRAM_PROFILE,15,none,none,none,none,none,none,tmpz5f94_sd.fna&_nc_gid=spdEaB2Lb0gw966KoBaCBA...


# YouTube

In [1]:
import json
import pandas as pd
from datetime import datetime

# Load JSON data
with open('tiktok_poloshirts.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Flatten nested structures
# json_normalize will automatically expand nested dicts; lists remain as lists
df = pd.json_normalize(
    data,
    sep='.'  # use dot to separate nested keys
)

# Drop columns with all missing values
df.dropna(axis=1, how='all', inplace=True)

# Convert Unix timestamps to datetime for start_date and end_date
# for col in ['start_date', 'end_date']:
#     if col in df.columns:
#         df[col] = pd.to_datetime(df[col], unit='s', errors='coerce')

# Handle missing values
# For numeric columns, fill missing with 0
num_cols = df.select_dtypes(include=['number']).columns
df[num_cols] = df[num_cols].fillna(0)

# For object/string columns, fill missing with empty string
obj_cols = df.select_dtypes(include=['object']).columns
df[obj_cols] = df[obj_cols].fillna('')

# Example: if there are list columns you may want to convert them to strings
list_cols = [c for c in df.columns if df[c].apply(lambda x: isinstance(x, list)).any()]
for col in list_cols:
    df[col] = df[col].apply(lambda x: ','.join(map(str, x)) if isinstance(x, list) else x)

# Save to CSV
output_csv = 'youtube_500_shorts_poloshirts_clean.csv'
df.to_csv(output_csv, index=False, encoding='utf-8')

print(f"Cleaned CSV saved to {output_csv}")


Cleaned CSV saved to youtube_500_shorts_poloshirts_clean.csv


In [2]:
yt = pd.read_csv('youtube_500_shorts_poloshirts_clean.csv')

In [3]:
print(yt.head())

            id                                              title  \
0  GLbplfA44tE  Get Ready With Me 19 #poloshirts #polo #outfit...   
1  C2QYn82LcpE                                    ORN Silverswift   
2  gIxzpPnh8yM  Styling blue shirt .Styling white trousers. #f...   
3  rTDlsDjnOPs  Polo Shirt tuck in hack #poloshirts #hack #pol...   
4  cGDN-XX8OhI                        TRESPASS Rygsæk model Albus   

                                          url  viewCount  \
0  https://www.youtube.com/shorts/GLbplfA44tE        247   
1  https://www.youtube.com/shorts/C2QYn82LcpE         75   
2  https://www.youtube.com/shorts/gIxzpPnh8yM       2900   
3  https://www.youtube.com/shorts/rTDlsDjnOPs   29000000   
4  https://www.youtube.com/shorts/cGDN-XX8OhI        540   

                                           fromYTUrl  \
0  https://www.youtube.com/hashtag/poloshirts/shorts   
1  https://www.youtube.com/hashtag/poloshirts/shorts   
2  https://www.youtube.com/hashtag/poloshirts/shorts   


In [7]:
columns_to_keep = ['id','title','url','viewCount','thumbnailUrl','date']

# Filter the DataFrame
yt = yt[columns_to_keep]

# 2. Drop rows where snapshot.videos is NaN
yt = yt.dropna(subset=['url'])


In [18]:
# Filter DataFrame
filtered = yt[yt['viewCount'] >= 4000]

# Convert filtered DataFrame to list of dicts
records = filtered.to_dict(orient='records')

# Save to JSON file with pretty formatting
with open('yt_filtered.json', 'w', encoding='utf-8') as f:
    json.dump(records, f, indent=4, ensure_ascii=False)

# TikTok

In [24]:
import json
import pandas as pd
from datetime import datetime

# Load JSON data
with open('tiktok_poloshirts_500.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Flatten nested structures
# json_normalize will automatically expand nested dicts; lists remain as lists
df = pd.json_normalize(
    data,
    sep='.'  # use dot to separate nested keys
)

# Drop columns with all missing values
df.dropna(axis=1, how='all', inplace=True)

# Convert Unix timestamps to datetime for start_date and end_date
# for col in ['start_date', 'end_date']:
#     if col in df.columns:
#         df[col] = pd.to_datetime(df[col], unit='s', errors='coerce')

# Handle missing values
# For numeric columns, fill missing with 0
num_cols = df.select_dtypes(include=['number']).columns
df[num_cols] = df[num_cols].fillna(0)

# For object/string columns, fill missing with empty string
obj_cols = df.select_dtypes(include=['object']).columns
df[obj_cols] = df[obj_cols].fillna('')

# Example: if there are list columns you may want to convert them to strings
list_cols = [c for c in df.columns if df[c].apply(lambda x: isinstance(x, list)).any()]
for col in list_cols:
    df[col] = df[col].apply(lambda x: ','.join(map(str, x)) if isinstance(x, list) else x)

# Save to CSV
output_csv = 'tiktok_poloshirts_clean.csv'
df.to_csv(output_csv, index=False, encoding='utf-8')

print(f"Cleaned CSV saved to {output_csv}")


Cleaned CSV saved to tiktok_poloshirts_clean.csv


In [25]:
tk = pd.read_csv('tiktok_poloshirts_clean.csv')
tk

,id,text,textLanguage,createTime,createTimeISO,isAd,isMuted,webVideoUrl,mediaUrls,diggCount,...,videoMeta.format,searchHashtag.views,searchHashtag.name,locationMeta.address,locationMeta.city,locationMeta.cityCode,locationMeta.countryCode,locationMeta.locationName,locationMeta.locationId,videoMeta.subtitleLinks
0,7449699136917556487,Oldmoney style kinda have this in thier closet...,en,1734518250,2024-12-18T10:37:30.000Z,False,False,https://www.tiktok.com/@oldfitman.kh/video/744...,NaN,39900,...,mp4,384500000,poloshirts,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7486021209092771079,#thednsclothing #mensfashion #fyp #summermenst...,un,1742975142,2025-03-26T07:45:42.000Z,False,False,https://www.tiktok.com/@thednsclothing/video/7...,NaN,10400,...,mp4,384500000,poloshirts,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7413801418215230728,White Full Sleeve Polo Shirt Complete order ...,en,1726160162,2024-09-12T16:56:02.000Z,False,False,https://www.tiktok.com/@blackstreet_official/v...,NaN,11700,...,mp4,384500000,poloshirts,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7424268621280496903,ganda ng fit💙🖤 #womenpoloshirt #poloshirt #nav...,tl,1728597247,2024-10-10T21:54:07.000Z,True,False,https://www.tiktok.com/@recosbymariee/video/74...,NaN,1618,...,mp4,384500000,poloshirts,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7387050836402244896,✨popular outfit✨ #foryoupage #lanadelrey #fory...,en,1719931808,2024-07-02T14:50:08.000Z,False,False,https://www.tiktok.com/@thilo.k_/video/7387050...,NaN,1100000,...,mp4,384500000,poloshirts,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,7503645007203650862,W polo #mensfashion #mensfashioninspo #menoutf...,pl,1747078530,2025-05-12T19:35:30.000Z,False,False,https://www.tiktok.com/@branttakes/video/75036...,NaN,18,...,mp4,384500000,poloshirts,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,7503573536741641518,Very comfy and stylish #mvc55 #shirt #shirts #...,en,1747061907,2025-05-12T14:58:27.000Z,False,False,https://www.tiktok.com/@dave_mvc55/video/75035...,NaN,11,...,mp4,384500000,poloshirts,NaN,NaN,NaN,NaN,NaN,NaN,"{'language': 'rus-RU', 'downloadLink': 'https:..."
97,7491223180233821447,Rekomended buat yg lagi cari polo shirt ✨⚜️ #o...,id,1744186327,2025-04-09T08:12:07.000Z,True,False,https://www.tiktok.com/@wrizkian/video/7491223...,NaN,2802,...,mp4,384500000,poloshirts,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,7493126680609918215,POV: you discover the best knitted polo for un...,en,1744629513,2025-04-14T11:18:33.000Z,False,False,https://www.tiktok.com/@1202damani/video/74931...,NaN,2239,...,mp4,384500000,poloshirts,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
tk.dropna(axis=1, how='all', inplace=True)
tk.nunique()

id                                  100
text                                100
textLanguage                         14
createTime                          100
createTimeISO                       100
isAd                                  2
isMuted                               1
webVideoUrl                         100
diggCount                            98
shareCount                           97
playCount                            95
collectCount                         99
commentCount                         78
mentions                              6
detailedMentions                      6
hashtags                             94
effectStickers                        4
isSlideshow                           1
isPinned                              1
isSponsored                           2
input                                 1
authorMeta.id                        91
authorMeta.name                      91
authorMeta.profileUrl                91
authorMeta.nickName                  91


In [ ]:
tk['']

0     https://www.tiktok.com/@oldfitman.kh/video/744...
1     https://www.tiktok.com/@thednsclothing/video/7...
2     https://www.tiktok.com/@blackstreet_official/v...
3     https://www.tiktok.com/@recosbymariee/video/74...
4     https://www.tiktok.com/@thilo.k_/video/7387050...
                            ...                        
95    https://www.tiktok.com/@branttakes/video/75036...
96    https://www.tiktok.com/@dave_mvc55/video/75035...
97    https://www.tiktok.com/@wrizkian/video/7491223...
98    https://www.tiktok.com/@1202damani/video/74931...
99    https://www.tiktok.com/@chorchonr/video/746407...
Name: webVideoUrl, Length: 100, dtype: object

In [31]:
tk['']

0     Oldmoney style kinda have this in thier closet...
1     #thednsclothing #mensfashion #fyp #summermenst...
2     White Full Sleeve Polo Shirt  Complete order  ...
3     ganda ng fit💙🖤 #womenpoloshirt #poloshirt #nav...
4     ✨popular outfit✨ #foryoupage #lanadelrey #fory...
                            ...                        
95    W polo #mensfashion #mensfashioninspo #menoutf...
96    Very comfy and stylish #mvc55 #shirt #shirts #...
97    Rekomended buat yg lagi cari polo shirt ✨⚜️ #o...
98    POV: you discover the best knitted polo for un...
99    polo shirt brown  มาถึงสีน้ำตาลแล้ว 🤎  *ไม่อนุ...
Name: text, Length: 100, dtype: object

In [32]:
columns_to_keep = ['id','text','createTimeISO','webVideoUrl','diggCount','shareCount','playCount',
                   'collectCount','hashtags','authorMeta.name','authorMeta.profileUrl','authorMeta.nickName',
                   'authorMeta.following','authorMeta.fans','authorMeta.heart','musicMeta.playUrl','videoMeta.subtitleLinks']

# Filter the DataFrame
tk = tk[columns_to_keep]
tk

,id,text,createTimeISO,webVideoUrl,diggCount,shareCount,playCount,collectCount,hashtags,authorMeta.name,authorMeta.profileUrl,authorMeta.nickName,authorMeta.following,authorMeta.fans,authorMeta.heart,musicMeta.playUrl,videoMeta.subtitleLinks
0,7449699136917556487,Oldmoney style kinda have this in thier closet...,2024-12-18T10:37:30.000Z,https://www.tiktok.com/@oldfitman.kh/video/744...,39900,3393,1500000,6076,"{'id': '34231', 'name': 'polo', 'title': '', '...",oldfitman.kh,https://www.tiktok.com/@oldfitman.kh,Oldfitman.kh,0,4652,135200,https://v77.tiktokcdn.com/1e5b2d46546d94c89bc1...,NaN
1,7486021209092771079,#thednsclothing #mensfashion #fyp #summermenst...,2025-03-26T07:45:42.000Z,https://www.tiktok.com/@thednsclothing/video/7...,10400,1075,334100,3071,"{'id': '7239336408186355719', 'name': 'thednsc...",thednsclothing,https://www.tiktok.com/@thednsclothing,The d&s clothing,426,7125,94400,https://sf16-ies-music-va.tiktokcdn.com/obj/to...,NaN
2,7413801418215230728,White Full Sleeve Polo Shirt Complete order ...,2024-09-12T16:56:02.000Z,https://www.tiktok.com/@blackstreet_official/v...,11700,825,666600,1803,"{'id': '34231', 'name': 'polo', 'title': '', '...",blackstreet_official,https://www.tiktok.com/@blackstreet_official,Black Street,2,990600,8400000,https://sf16-ies-music-va.tiktokcdn.com/obj/to...,NaN
3,7424268621280496903,ganda ng fit💙🖤 #womenpoloshirt #poloshirt #nav...,2024-10-10T21:54:07.000Z,https://www.tiktok.com/@recosbymariee/video/74...,1618,103,281700,407,"{'id': '7158126301381918747', 'name': 'womenpo...",recosbymariee,https://www.tiktok.com/@recosbymariee,recosbymariee🧿🍀🧚🏻‍♀️,217,18100,449300,https://v77.tiktokcdn.com/ecfbc83397abf5f9a47e...,NaN
4,7387050836402244896,✨popular outfit✨ #foryoupage #lanadelrey #fory...,2024-07-02T14:50:08.000Z,https://www.tiktok.com/@thilo.k_/video/7387050...,1100000,86200,10100000,92600,"{'id': '88764338', 'name': 'foryoupage', 'titl...",thilo.k_,https://www.tiktok.com/@thilo.k_,Thilo.k_✨🐟,95,111400,5100000,https://v77.tiktokcdn.com/605e7da5841e3c1e15d4...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,7503645007203650862,W polo #mensfashion #mensfashioninspo #menoutf...,2025-05-12T19:35:30.000Z,https://www.tiktok.com/@branttakes/video/75036...,18,0,596,1,"{'id': '336181', 'name': 'mensfashion', 'title...",branttakes,https://www.tiktok.com/@branttakes,Brant 🩶,191,43700,2900000,https://v19.tiktokcdn-us.com/3d54b0d271c744d50...,NaN
96,7503573536741641518,Very comfy and stylish #mvc55 #shirt #shirts #...,2025-05-12T14:58:27.000Z,https://www.tiktok.com/@dave_mvc55/video/75035...,11,1,2076,0,"{'id': '1686987239676933', 'name': 'mvc55', 't...",dave_mvc55,https://www.tiktok.com/@dave_mvc55,Dave’s,6889,323900,34900000,https://v19.tiktokcdn-us.com/93daa654060b5aa8c...,"{'language': 'rus-RU', 'downloadLink': 'https:..."
97,7491223180233821447,Rekomended buat yg lagi cari polo shirt ✨⚜️ #o...,2025-04-09T08:12:07.000Z,https://www.tiktok.com/@wrizkian/video/7491223...,2802,519,357900,1485,"{'id': '3958603', 'name': 'outfitinspo', 'titl...",wrizkian,https://www.tiktok.com/@wrizkian,rizkiann,50,18200,2300000,https://v19.tiktokcdn-us.com/341663fcb8b8ea8e8...,NaN
98,7493126680609918215,POV: you discover the best knitted polo for un...,2025-04-14T11:18:33.000Z,https://www.tiktok.com/@1202damani/video/74931...,2239,58,71200,390,"{'id': '229207', 'name': 'fyp', 'title': '', '...",1202damani,https://www.tiktok.com/@1202damani,Damani♣️,54,8419,234900,https://sf19.tiktokcdn-us.com/obj/tos-alisg-ve...,NaN


In [26]:
import json
import pandas as pd

# 1. Load the JSON records
with open('tiktok_polo_shirts_ads.json', 'r') as f:
    data = json.load(f)

# 2. Normalize into a flat DataFrame
df = pd.json_normalize(data)

# 3. Filter to keep only rows where videoUrl is not null or NaN
if 'adVideoUrl' in df.columns:
    df = df[df['adVideoUrl'].notna() & df['adVideoUrl'].astype(str).str.strip().ne('')]

# 4. Convert epoch milliseconds to datetime
for col in ['adStartDate', 'adEndDate']:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], unit='ms')

# 5. (Optional) Expand targeting lists into JSON strings
for col in ['targetingByLocation', 'targetingByAge', 'targetingByGender']:
    if col in df.columns:
        df[col] = df[col].apply(lambda lst: json.dumps(lst, ensure_ascii=False))

# 6. Write out to CSV
csv_path = 'tiktok_polo_shirts_ads_filtered.csv'
df.to_csv(csv_path, index=False)
print(f"Wrote filtered data to {csv_path!r}")

# 7. Analyze all columns
print("\n=== DataFrame Info ===")
df.info()

print("\n=== Missing values per column ===")
print(df.isna().sum())

print("\n=== Top 5 values in each column ===")
for col in df.columns:
    print(f"\n-- {col!r} --")
    try:
        print(df[col].value_counts(dropna=False).head(5))
    except Exception as e:
        print(f"  (could not compute value_counts: {e})")

print("\n=== Summary statistics for numeric columns ===")
print(df.select_dtypes(include='number').describe())

print("\n=== Summary for datetime columns ===")
for col in df.select_dtypes(include='datetime').columns:
    print(f"{col}: min = {df[col].min()}, max = {df[col].max()}, count = {df[col].count()}")


Wrote filtered data to 'tiktok_polo_shirts_ads_filtered.csv'

=== DataFrame Info ===
<class 'pandas.core.frame.DataFrame'>
Index: 37 entries, 0 to 58
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   adId                 37 non-null     object        
 1   adTitle              37 non-null     object        
 2   adType               37 non-null     object        
 3   adSpent              37 non-null     object        
 4   adVideoUrl           37 non-null     object        
 5   adVideoCover         37 non-null     object        
 6   adStartDate          37 non-null     datetime64[ns]
 7   adEndDate            37 non-null     datetime64[ns]
 8   advertiserId         37 non-null     object        
 9   advertiserName       37 non-null     object        
 10  adImpressions        37 non-null     object        
 11  advertiserPaidForBy  37 non-null     object        
 12  adTotalRegions

In [17]:
import json
import pandas as pd

# 1. Load the JSON array
with open('youtube_polo_shirts_ads.json', 'r') as f:
    data = json.load(f)

# 2. Flatten into a DataFrame
df = pd.json_normalize(data)

# 3. Convert ISO‑format date strings to datetime
if 'date' in df.columns:
    df['date'] = pd.to_datetime(df['date'])

# 4. Serialize nested list/object columns as JSON strings
for col in ['descriptionLinks', 'hashtags', 'formats', 'subtitles', 'comments']:
    if col in df.columns:
        df[col] = df[col].apply(
            lambda x: json.dumps(x, ensure_ascii=False) 
                      if isinstance(x, (list, dict)) 
                      else x
        )

df = df[(df['likes'] > 500) & (df['viewCount'] > 8000)]
# 5. Export to CSV
csv_path = 'youtube_filtered_polo_shirts_ads.csv'
df.to_csv(csv_path, index=False)
print(f"Wrote flattened data to {csv_path!r}")

# 6. Analyze all columns
print("\n=== DataFrame Info ===")
df.info()

print("\n=== Missing values per column ===")
print(df.isna().sum())

print("\n=== Top 5 values in each column ===")
for col in df.columns:
    print(f"\n-- {col!r} --")
    try:
        print(df[col].value_counts(dropna=False).head(5))
    except Exception as e:
        print(f"  (could not compute value_counts: {e})")

print("\n=== Summary statistics for numeric columns ===")
print(df.select_dtypes(include='number').describe())

print("\n=== Summary for datetime columns ===")
for col in df.select_dtypes(include='datetime').columns:
    print(f"{col}: min = {df[col].min()}, max = {df[col].max()}, count = {df[col].count()}")


Wrote flattened data to 'youtube_filtered_polo_shirts_ads.csv'

=== DataFrame Info ===
<class 'pandas.core.frame.DataFrame'>
Index: 69 entries, 0 to 149
Data columns (total 28 columns):
 #   Column               Non-Null Count  Dtype              
---  ------               --------------  -----              
 0   title                69 non-null     object             
 1   type                 69 non-null     object             
 2   id                   69 non-null     object             
 3   url                  69 non-null     object             
 4   thumbnailUrl         69 non-null     object             
 5   viewCount            69 non-null     int64              
 6   date                 69 non-null     datetime64[ns, UTC]
 7   likes                69 non-null     int64              
 8   location             8 non-null      object             
 9   channelName          69 non-null     object             
 10  channelUrl           69 non-null     object             
 11  cha

In [18]:
data = pd.read_csv('youtube_filtered_polo_shirts_ads.csv')
data.head(30)

,title,type,id,url,thumbnailUrl,viewCount,date,likes,location,channelName,...,subtitles,order,commentsTurnedOff,comments,fromYTUrl,isMonetized,hashtags,formats,isMembersOnly,input
0,Are Ralph Lauren Polos Worth It? Iconic Preppy...,video,-h7T4N1z1lA,https://www.youtube.com/watch?v=-h7T4N1z1lA,https://i.ytimg.com/vi/-h7T4N1z1lA/maxresdefau...,98476,2023-07-17 15:00:13+00:00,2100,NaN,Gentleman's Gazette,...,NaN,7,False,NaN,https://www.youtube.com/results?search_query=p...,NaN,[],[],False,polo shirts
1,Stop Wearing Polo Shirts WRONG! (Top 10 Polo W...,video,8uQysrNE_I8,https://www.youtube.com/watch?v=8uQysrNE_I8,https://i.ytimg.com/vi/8uQysrNE_I8/maxresdefau...,184206,2023-05-20 13:09:41+00:00,3300,NaN,alpha m.,...,NaN,2,False,NaN,https://www.youtube.com/results?search_query=p...,NaN,[],[],False,polo shirts
2,Which Brand Makes The BEST POLO? (Ralph Lauren...,video,yCqR054y1xM,https://www.youtube.com/watch?v=yCqR054y1xM&t=92s,https://i.ytimg.com/vi/yCqR054y1xM/maxresdefau...,175856,2023-05-03 09:30:14+00:00,2200,NaN,Men's Fashion Files,...,NaN,4,False,NaN,https://www.youtube.com/results?search_query=p...,NaN,[],[],False,polo shirts
3,"Polo Shirts: Your Guide to Buying, Styling, Hi...",video,YGASQewRYAo,https://www.youtube.com/watch?v=YGASQewRYAo,https://i.ytimg.com/vi/YGASQewRYAo/maxresdefau...,82375,2023-05-26 15:00:45+00:00,2300,NaN,Gentleman's Gazette,...,NaN,0,False,NaN,https://www.youtube.com/results?search_query=p...,NaN,[],[],False,polo shirts
4,How to Tell If Your Polo Shirts Fit | Polo Shi...,video,MNHba2ebWWM,https://www.youtube.com/watch?v=MNHba2ebWWM,https://i.ytimg.com/vi/MNHba2ebWWM/maxresdefau...,104461,2016-08-20 21:01:15+00:00,1100,NaN,Brock McGoff,...,NaN,50,False,NaN,https://www.youtube.com/results?search_query=p...,NaN,[],[],False,polo shirts
5,How Should A Polo Shirt Fit? - Men's Clothing ...,video,M_-D6C5K8h8,https://www.youtube.com/watch?v=M_-D6C5K8h8,https://i.ytimg.com/vi/M_-D6C5K8h8/maxresdefau...,230928,2016-02-25 14:00:00+00:00,2500,NaN,Mens Fashioner,...,NaN,52,False,NaN,https://www.youtube.com/results?search_query=p...,NaN,[],[],False,polo shirts
6,Which Lacoste Polo Fits You The Best! | Mens P...,video,FbUTNh-JcqU,https://www.youtube.com/watch?v=FbUTNh-JcqU,https://i.ytimg.com/vi/FbUTNh-JcqU/maxresdefau...,61605,2022-06-19 08:00:03+00:00,512,NaN,Ranga Don,...,NaN,55,False,NaN,https://www.youtube.com/results?search_query=p...,NaN,[],[],False,polo shirts
7,FAKE Polo Ralph Lauren ????,shorts,Qih3KWys73M,https://www.youtube.com/shorts/Qih3KWys73M,https://i.ytimg.com/vi/Qih3KWys73M/maxresdefau...,51519,2025-03-06 21:15:55+00:00,751,NaN,SurfsUpFinds,...,NaN,63,False,NaN,https://www.youtube.com/results?search_query=p...,NaN,[],[],False,polo shirts
8,Folding techniques for Polo Shirt#shortvideo #...,shorts,8pc12Us7pHI,https://www.youtube.com/shorts/8pc12Us7pHI,https://i.ytimg.com/vi/8pc12Us7pHI/maxresdefau...,109536,2023-03-30 18:39:17+00:00,1700,NaN,Sui Dhaga,...,NaN,80,False,NaN,https://www.youtube.com/results?search_query=p...,NaN,[],[],False,polo shirts
9,3-Second Polo Shirt Fold! Master the Technique,shorts,1AVx5utG09I,https://www.youtube.com/shorts/1AVx5utG09I,https://i.ytimg.com/vi/1AVx5utG09I/maxresdefau...,42505,2024-08-21 00:31:41+00:00,724,NaN,Beauty Studio 77,...,NaN,71,False,NaN,https://www.youtube.com/results?search_query=p...,NaN,"[""#poloshirt"", ""#fashionhacks"", ""#shorts""]",[],False,polo shirts
